In [1]:
import math
import pathlib

from compas.datastructures import Mesh
from compas.files import OBJ
from compas.geometry import Frame
from compas.geometry import Rotation
from compas_model.elements import BlockElement
from compas_model.models import Model

filepath = pathlib.Path().home() / "Code" / "compas_model" / "data" / "wall_stack.obj"

obj = OBJ(filepath)
obj.read()

meshes = []
for name in obj.objects:  # type: ignore
    vertices, faces = obj.objects[name]  # type: ignore
    mesh = Mesh.from_vertices_and_faces(vertices, faces)
    mesh.name = name
    meshes.append(mesh)

model = Model()

supports = model.add_group(name="supports")
blocks = model.add_group(name="blocks")

block = BlockElement(shape=meshes[0], is_support=True)
model.add_element(block, parent=supports)

for mesh in meshes[1:]:
    block = BlockElement(shape=mesh)
    model.add_element(block, parent=blocks)

R = Rotation.from_axis_and_angle(axis=[0, 0, 1], angle=math.radians(30))

blocks.frame = Frame.worldXY()
blocks.frame.transform(R)

blocks.children[-1].element.transformation = R


In [2]:
from compas_model.algorithms import get_collision_pairs

elements = list(model.elements())
interfaces = []
for item in get_collision_pairs(model, tolerance_area=0.1):
    i, j, collisions = item
    for faces, polygon in collisions:
        interfaces.append(polygon)
        model.add_interaction(elements[i], elements[j])

In [3]:
print(model)

Spatial Hierarchy
<Tree with 9 nodes>
Element Interactions
<Graph with 6 nodes, 5 edges>
0
- 1: None
1
- 0: None
- 2: None
2
- 1: None
- 3: None
3
- 2: None
- 4: None
4
- 3: None
- 5: None
5
- 4: None

Element Groups
n/a



In [4]:
from compas.colors import Color
from compas_notebook.viewer import Viewer

viewer = Viewer()

viewer.scene.add(model,
    show_elements=True,
    show_element_faces=False,
    show_faces=False,
    show_edges=True,
    show_vertices=True,
    vertexsize=0.1
)

# for element in model.elements():
#     viewer.scene.add(element.geometry, color=Color.from_hex('#cccccc'), show_faces=False, show_edges=True)

# for polygon in interfaces:
#     viewer.scene.add(polygon, color=Color.red())

viewer.show()

PyThreeJS Model elements registered.
PyThreeJS SceneObjects registered.
TNA Notebook SceneObjects registered.
